In [1]:
import pandas as pd
import numpy as np

dataset_20_21 = '../web_scraping/matches.csv'
data = pd.read_csv(dataset_20_21)
data_df = data[["date", "venue", "team", "opponent", "gf", "ga", "result", "xg", "sot","poss"]]
data_df["date"] = pd.to_datetime(data_df["date"], format="%Y-%m-%d")
data_df["opponent"] = data_df["opponent"].str.replace("Utd", "United").replace("West Ham", "West Ham United").replace("Nott'ham Forest", "Nottingham Forest").replace("Tottenham", "Tottenham Hotspur").replace("Wolves", "Wolverhampton Wanderers").replace("Brighton", "Brighton and Hove Albion").replace("West Brom", "West Bromwich Albion");
data_df["Score"] = data_df["gf"] -  data_df["ga"]

# data_df["Round"] = np.zeros((len(data_df), 1))
# current_round = 1
# for row in range(1,len(data)+1):
#   data_df['Round'][row-1] = current_round
#   if (row % 10) == 0:
#     current_round += 1

teamsList= {k:v for v, k in enumerate(np.sort(data_df["team"].unique()))}
data_df["teamId"] = [teamsList[id] for id in list(data_df["team"].values)]
data_df["opponentId"] = [teamsList[id] for id in list(data_df["opponent"].values)]
data_df["duplicate"] = False

data_df = data_df.sort_values(by=["date"], ascending=False)
data_df = data_df.loc[(data_df["teamId"] == 12) | (data_df["opponentId"] == 12)]



for i in range(len(data_df)):
  fixture = data_df.iloc[i]
  if(fixture["duplicate"] == False):
    duplicate = data_df.index[(data_df["date"] == fixture["date"]) & (data_df["teamId"] == fixture["opponentId"]) & (data_df["opponentId"] == fixture["teamId"])]
    if(data_df.at[duplicate[0], "duplicate"] == False):
        data_df.at[duplicate[0], "duplicate"] = True

data_df= data_df.loc[data_df["duplicate"] == False]




C:\Users\user\AppData\Local\Temp\ipykernel_17988\3832146072.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df["date"] = pd.to_datetime(data_df["date"], format="%Y-%m-%d")
C:\Users\user\AppData\Local\Temp\ipykernel_17988\3832146072.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df["opponent"] = data_df["opponent"].str.replace("Utd", "United").replace("West Ham", "West Ham United").replace("Nott'ham Forest", "Nottingham Forest").replace("Tottenham", "Tottenham Hotspur").replace("Wolves", "

In [7]:
import torch
def get_last_5_matches(row):
    last_matches = data_df.loc[data_df["date"] < row["date"]].head(5);    
    array = last_matches[["gf","ga","xg", "sot", "poss"]].to_numpy();
    if(len(last_matches) <= 5):
        base = np.zeros((5,5))
        for i in range(len(last_matches)):
            base[i,:] = last_matches.iloc[i][["gf","ga","xg", "sot", "poss"]].to_numpy()
        array = base
    row["last_5_match_stats"] = array.tolist()
    return row   

def convert_result_to_one_hot(result):
    if result == "W":
        return [1,0,0]
    elif result == "D":
        return [0,1,0]
    else:
        return [0,0,1]
    
data_df["new_result"] = data_df["result"].apply(convert_result_to_one_hot)
data_df= data_df.apply(get_last_5_matches, axis=1)

data_df.head()

# data_df[["date","team","opponent", "last_5_matches"]]
final_df = data_df[[ "gf", "ga", "xg", "sot", "poss","last_5_match_stats", "new_result"]]
final_df = final_df.rename(columns={"new_result": "result"})
final_df.dtypes

gf                      int64
ga                      int64
xg                    float64
sot                   float64
poss                  float64
last_5_match_stats     object
result                 object
dtype: object

In [ ]:
from FootballMatchPredictionModel import FootballMatchPredictionModel
from FootballMatchDataset import FootballMatchDataset
import torch.optim as optim
import torch.utils.data as data
from torch.utils.data import DataLoader, random_split


dataset = FootballMatchDataset(final_df)
train, test, val = random_split(dataset, [0.7,0.2,0.1])

train_loader = data.DataLoader(train, batch_size=10)
val_loader = data.DataLoader(val, batch_size=10)
test_loader = data.DataLoader(test, batch_size=10)
print(train_loader)

model = FootballMatchPredictionModel(final_df.shape[1]);
optimiser = optim.Adam(model.parameters())
loss_function = torch.nn.MSELoss()

num_epochs = 1
for epoch in range(num_epochs):
    batch_id = 0
    model.train(True)
    for batch in train_loader:
        #split
        features,labels = batch
        #reset
        optimiser.zero_grad()
        #predict
        prediction = model(features)
        #calculate loss
        criterion = loss_function(prediction, labels)
        #backpropogate
        criterion.backward()
        #update weights
        optimiser.step()
        batch_id += 1
        print(f"completed: {batch_id}")
    #test
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            features, labels = data
            features, labels = features.to(device), labels.to(device)
            outputs = model(features)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    result = correct / total
    print("Accuracy: ", result)

